# Topic Modelling
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [8]:
import pandas as pd
import nltk
import json
from pathlib import Path
import glob
import math
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [9]:
stop_words = stopwords.words('german')

## Load Data

In [29]:
records = []
for f in glob.glob(str(Path('../export/tags/*.json'))):
    records = records + json.load(open(f, 'r', encoding='utf-8'))

print(len(records))

46521


In [30]:
# Members only, no Presidents
r_members = list(filter(lambda x: x['ismember'] == True, records))
r_members = list(filter(lambda x: x['funktion'] not in ['Präsidium', '2. Vizepräsidium', '1. Vizepräsidium'], r_members))
print(len(r_members))

34497


## Prepare Data

In [42]:
# Remove Stopwords and only keep noun, adj, vb and adv

keep = ['NN', 'NE',
    'ADJA', 'ADJD',
    'VMFIN', 'VAFIN', 'VVFIN', 'VAIMP', 'VVIMP', 'VVINF', 'VAINF', 'VMINF', 'VVIZU', 'VVPP', 'VMPP', 'VAPP',
    'ADV']

for votum in r_members:
    # Reversed Loop, we will remove items from the list during loop!
    for i in reversed(range(0, len(votum['tags']))):
        tag = votum['tags'][i]
        if tag[0].lower() in stop_words:

            # Stopword found. Remove
            votum['tags'].remove(tag)

        elif tag[2] not in keep:
            
            # Not tag we need. Remove
            votum['tags'].remove(tag)

In [65]:
# Stick the words together to get a nice list for our corpora and split by sex
data_m = []
data_w = []
for votum in r_members:
    if votum['geschlecht'] == 'm':
        data_m.append(list(map(lambda tag: tag[0], votum['tags'])))
    else:
        data_w.append(list(map(lambda tag: tag[0], votum['tags'])))

# Build Dictionary
id2word_m = corpora.Dictionary(data_m)
id2word_w = corpora.Dictionary(data_w)

# Term Document Frequency
corpus_m = [id2word_m.doc2bow(text) for text in data_m]
corpus_w = [id2word_w.doc2bow(text) for text in data_w]

## Go LDA

In [66]:
lda_model_m = gensim.models.ldamodel.LdaModel(corpus=corpus_m, id2word=id2word_m, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

IndexError: index 164476 is out of bounds for axis 1 with size 164476

In [ ]:
lda_model_w = gensim.models.ldamodel.LdaModel(corpus=corpus_m, id2word=id2word_w, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [56]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [58]:
corpus[:1]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 3),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 2),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 2),
  (51, 1),
  (52, 1)]]